In [ ]:
!pip install python-terrier

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 102 kB 6.1 MB/s 
     |████████████████████████████████| 1.3 MB 31.2 MB/s 
     |████████████████████████████████| 69 kB 6.7 MB/s 
     |████████████████████████████████| 311 kB 62.5 MB/s 
     |████████████████████████████████| 46 kB 4.4 MB/s 
     |████████████████████████████████| 286 kB 55.0 MB/s 
     |████████████████████████████████| 45 kB 3.2 MB/s 
     |████████████████████████████████| 1.2 MB 48.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 126 kB 57.2 MB/s 
     |████████████████████████████████| 72 kB 1.6 MB/s 
     |████████████████████████████████| 294 kB 43.8 MB/s 
  Created wheel for python-terrier: filename=python_terrier-0.9.1-py3-none-any.whl size=110767 sha256=8c44793ca0f278917cefc2c0a682c0cdee25336f043f

In [ ]:
import pandas as pd
# Helpful for showing indexing information
pd.set_option('display.max_colwidth', 150)

import pyterrier as pt
import os
import random
import numpy as np
from sklearn.metrics import ndcg_score

In [ ]:
if not pt.started():
    pt.init()

terrier-assemblies 5.7 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.7 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.9.1 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7



In [ ]:
qrels = pd.read_csv('final_results_references - final_results_references.csv', usecols=["qid", "docno", "Relevance (1-5)"], header=0)
print(qrels.head(4))
qrels.columns = ['qid', 'docno', 'label'] 
# qrels['label'] = qrels['label'].replace([1, 2], 0)
# qrels['label'] = qrels['label'].replace([3, 4, 5], 1)

qrels['qid']=qrels['qid'].values.astype(str)
print(qrels.info())
# df.name(columns={'qid': 'query-id', 'docno': 'corpus-id', "Relevance (1-5)": "score"}, inplace=True)
print(qrels.head(4))

   qid  docno  Relevance (1-5)
0    1  d1082                1
1    1  d1093                3
2    1  d1112                1
3    1  d1115                3
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3297 entries, 0 to 3296
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   qid     3297 non-null   object
 1   docno   3297 non-null   object
 2   label   3297 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 77.4+ KB
None
  qid  docno  label
0   1  d1082      1
1   1  d1093      3
2   1  d1112      1
3   1  d1115      3


In [ ]:
qrels_rand = pd.read_csv('final_results_references - final_results_references.csv', usecols=["qid", "docno", "Relevance (1-5)"], header=0)
# print(qrels.head(4))
qrels_rand['naive_predicted'] = [ random.randint(1,5)  for k in qrels_rand.index]
qrels_rand.columns = ['qid', 'docno', 'true_score', 'naive_predicted'] 
print(qrels_rand.head(20))

    qid  docno  true_score  naive_predicted
0     1  d1082           1                5
1     1  d1093           3                1
2     1  d1112           1                5
3     1  d1115           3                4
4     1  d1200           1                2
5     1  d1874           1                1
6     1  d2005           1                1
7     1   d209           1                4
8     1   d210           1                5
9     1   d218           1                1
10    1   d236           1                1
11    1  d2485           5                3
12    1  d2494           2                5
13    1   d284           1                1
14    1   d286           1                5
15    1   d311           1                4
16    1   d354           3                3
17    1   d379           2                3
18    1   d396           1                1
19    1   d424           1                4


In [ ]:
df = pd.read_csv('scrapped_amazon_general_queries_data.csv', usecols=["productTitle", "productDescription", "URL"], delimiter=';', header=0)
df['docno'] = 'd' + (df.reset_index().index).astype(str)
# df.columns = ["productTitle", "productDescription", "docno"] 
df['text'] = df[['productTitle', 'productDescription']].apply(lambda x: ' '.join(x[x.notnull()]), axis = 1)
print(df.head(5))

                                                                                                                                        productTitle  \
0                        Maybelline Hyper Easy Liquid Pen No-Skip Eyeliner Satin Finish Waterproof Formula Eye Liner Makeup Pitch Black 0.018 Fl; Oz   
1                         L'Oreal Paris Makeup Infallible Never Fail Original Mechanical Pencil Eyeliner with Built in Sharpener Black Brown 2 Count   
2  L'Oreal Paris L’Oreal Paris Makeup Infallible Grip Precision Felt Eyeliner Smudge Resistant Long Lasting Waterproof Eyeliner Grey Grey 0.03 fl oz   
3                                                                                             HotDeal Lakme Insta Deep Intensee Black Eye Liner 9 ml   
4                                                                                                      Lakme Insta Deep Intense Black Eye Liner 9 ml   

                                                                                       

In [ ]:
index_dir = os.path.abspath('amazonindex')
indexer = pt.DFIndexer(index_dir, overwrite=True)
index_ref = indexer.index(df["text"], df["docno"], )
index_ref.toString()

04:27:36.880 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (d181) - further warnings are suppressed
04:27:39.585 [main] WARN org.terrier.structures.indexing.Indexer - Indexed 2392 empty documents


'/content/amazonindex/data.properties'

In [ ]:
index = pt.IndexFactory.of(index_ref)

In [ ]:
print(index.getCollectionStatistics().toString())

Number of documents: 2801
Number of terms: 2070
Number of postings: 9807
Number of fields: 0
Number of tokens: 12273
Field names: []
Positions:   false



In [ ]:
br_bm25 = pt.BatchRetrieve(index, wmodel="BM25")

In [ ]:
qdf = pd.read_csv("combined_queries.csv", usecols=['qid', 'query'], header=0)
qdf['qid']=qdf['qid'].values.astype(str)
print(qdf.info())
print(qdf)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   qid     60 non-null     object
 1   query   60 non-null     object
dtypes: object(2)
memory usage: 1.1+ KB
None
   qid                                        query
0    1                       facewash for oily skin
1    2                   lip liner and lipstick set
2    3                                nude lipstick
3    4                        skin care sets & kits
4    5                             anti aging serum
5    6          vitamin c serum for face dark spots
6    7                        bronzer stick contour
7    8              mascara black volume and length
8    9            primer face makeup pore minimizer
9   10     moisturizer with spf for acne prone skin
10  11                  sunscreen for face dry skin
11  12                              concealer stick
12  13                 facewash w

In [ ]:
pt.Experiment(
    [br_bm25],
    topics=qdf,
    qrels=qrels,
    eval_metrics=["map", "ndcg", "ndcg_cut_5", "ndcg_cut_10"])

,name,map,ndcg,ndcg_cut_5,ndcg_cut_10
0,BR(BM25),0.582998,0.749079,0.735934,0.715005


In [ ]:
true_relevance = np.array(qrels_rand['true_score'])
true_relevance = np.array(sorted(true_relevance, reverse=True)).reshape(1,-1)
print(true_relevance)
# print(true_relevance)
scores = np.asarray([qrels_rand['naive_predicted']]).reshape(1,3297)
ndcg_score(true_relevance, scores)

[[5 5 5 ... 1 1 1]]


0.9053605972518237

In [ ]:
scores.shape

(1, 3297)

In [ ]:
np.array(qrels_rand['true_score'])

array([1, 3, 1, ..., 1, 3, 1])

In [ ]:
np.isnan(true_relevance)

TypeError: ignored

In [ ]:
print(true_relevance)

None
